In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.constants import G
from galpy.potential import (
    NFWPotential,
    ChandrasekharDynamicalFrictionForce,
    FDMDynamicalFrictionForce,
)
from galpy.orbit import Orbit

G = G.to(u.kpc**3 / (u.Msun * u.Gyr**2)).value

[Hui and al. 2017](https://arxiv.org/pdf/1610.08297) have derived the equations of quantum-mechanical treatment of the dynamical friction force. I propose here a new `Python` class to integrate orbits using these theoretical relations with `galpy`.

# Creating a dwarf galaxy dark matter halo potential

In [ ]:
M = 1e9  # Msol
rs = 1.57  # kpc
c = 18.4

A_NFW = np.log(1 + c) - c / (1 + c)

NFWHalo = NFWPotential(amp=M / A_NFW * u.Msun, a=rs * u.kpc)

# Initialize a globular cluster object

In [ ]:
Mobj = 1e6  # Msol
rhm = 1e-2  # kpc

### Dynamical friction

In [ ]:
cdf = ChandrasekharDynamicalFrictionForce(
    GMs=Mobj * u.Msun, rhm=rhm * u.kpc, dens=NFWHalo
)

# Initialize the orbit

In [ ]:
# initial conditions
R = 0.5
z = 0.0
phi = 0.0
Menclosed = NFWHalo.mass(R * u.kpc)
vR = 0.0
vT = np.sqrt(G * Menclosed / R)  # km/s
vz = 0.0

o = Orbit(
    [
        R * u.kpc,
        vR * u.km / u.s,
        vT * u.km / u.s,
        z * u.kpc,
        vz * u.km / u.s,
        phi * u.rad,
    ]
)
o_cdf = o()  # copy of orbit for dynamical friction
o_fdf = o()  # copy of orbit for fuzzy dynamical friction

# Integrate

In [ ]:
t = np.linspace(0, 1, 300) * u.Gyr
o.integrate(t, NFWHalo, method="odeint")
o_cdf.integrate(t, NFWHalo + cdf, method="odeint")

# Integrating orbits with Fuzzy Dark Matter (FDM) dynamical friction

The classical dynamical friction force used by `galpy` reads :

$$\vec{F}(\vec{x}, \vec{v}) = -4\pi \left[ \mathcal{G} M_{\rm obj}² \right] \left[ \mathcal{G} \rho(\vec{x}) \right] \ln \Lambda \left[ \mathrm{erf}(X) - \frac{2X}{\sqrt{\pi}} \exp \left( -X^2 \right) \right] \frac{\vec{v}}{|\vec{v}|^3}
$$
Using $X = \frac{|\vec{v}|}{\sqrt 2 \sigma(r)}$. And where 
$$\Lambda = \frac{r/\gamma}{\max(r_{\mathrm{hm}}, \mathcal{G}M/|\vec{v}|^2)}$$
Fuzzy dark matter model modifies the dynamical friction force according to the following relation : 
$$\vec{F}_\text{FDM} = -\frac{4\pi\mathcal{G}^2M_\text{obj}^2\rho}{v^3}C_{\rm FDM}(kr)\vec{v}$$
Where the coefficient $C_{\rm FDM}(kr)$ depends on $kr=\frac{mvr}{\hbar}$ and reads : 
$$C_{\rm FDM}(kr) = \text{Cin}(2kr)+\frac{\sin{(2kr)}}{2kr}-1$$
With
$$\text{Cin}(z) = \int_0^z\frac{1-\cos{(t)}}{t}\mathrm{d}t$$

Looking at the classical relation for dynamical friction, the equivalent of the coefficient $C(kr)$ in CDM scenario is : 
$$ C_{\rm CDM} =\ln \Lambda \left[ \mathrm{erf}(X) - \frac{2X}{\sqrt{\pi}} \exp \left( -X^2 \right) \right]$$

Since the FDM coefficient $C_{\rm FDM}(kr)$ has an asymptotic behavior, we must use the classical coefficient $C_{\rm CDM}$ as a cutoff whenever $C_{\rm FDM}(kr) \gt C_{\rm CDM}$, because it would mean that we are in the classical regime. This implemented in the class below, on the same model as `galpy` class `ChandrasekharDynamicalFrictionForce`.

In [ ]:
m = 3e-22  # mass of the fuzzy particle in eV
fdf = FDMDynamicalFrictionForce(
    GMs=Mobj * u.Msun, rhm=rhm * u.kpc, dens=NFWHalo, m=m * u.eV
)

In [ ]:
o_fdf.integrate(t, NFWHalo + fdf, method="odeint")

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.plot(o.x(t), o.y(t), label="No DF")
plt.plot(o_cdf.x(t), o_cdf.y(t), label="DF")
plt.plot(o_fdf.x(t), o_fdf.y(t), label="Fuzzy DF")
plt.xlabel("x [kpc]", fontsize=14, fontweight="bold")
plt.ylabel("y [kpc]", fontsize=14, fontweight="bold")
plt.gca().set_facecolor("whitesmoke")
plt.grid(True, which="both", linestyle="--", linewidth=0.7, alpha=0.7)
plt.legend(fontsize=12)

plt.subplot(122)
plt.plot(t, o.R(t), label="No DF")
plt.plot(t, o_cdf.R(t), label="DF")
plt.plot(t, o_fdf.R(t), label="Fuzzy DF")
plt.xlabel("t [Gyr]", fontsize=14, fontweight="bold")
plt.ylabel("R [kpc]", fontsize=14, fontweight="bold")
plt.legend(fontsize=12)
plt.tight_layout(pad=2.5)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.3)
plt.gca().set_facecolor("whitesmoke")
plt.grid(True, which="both", linestyle="--", linewidth=0.7, alpha=0.7)

plt.suptitle("Dynamical Friction in NFW halo", fontsize=16, fontweight="bold")
plt.show()

We can clearly see that Fuzzy dark matter dynamical friction is weaker than its classical version. 

# Analytical test

In the limit $kr\ll 1$ : the FDM coefficient reduces to $C(kr)\approx \frac{1}{3} (kr)^2= \frac{1}{3}(\frac{m}{\hbar})^2 v^2r^2$ 

Plugged into the FDM friction force : 
$$||\vec{F}_\text{FDM}|| =\frac{4\pi\mathcal{G}^2M_\text{obj}^2\rho(r)}{3}\left(\frac{m}{\hbar}\right)^2r^2 $$

Using isothermal sphere $ \rho(r) = \frac{v^2}{4\pi\mathcal{G}r^2}$ :
$$ ||\vec{F}_\text{FDM}|| = \mathcal{G}M_\text{obj}^2v^2\frac{1}{3}\left(\frac{m}{\hbar}\right)^2$$

The equation of motion derived from angular momentum derivative, assuming circular orbit with constant $v$, reads : 
$$ \frac{\mathrm{d}r}{\mathrm{dt}} = - \frac{\mathcal{G}M_\text{obj}v}{3}\left(\frac{m}{\hbar}\right)^2r$$

Which admit an analytical solution : 
$$ r(t) = r_0 e^{-t/\tau}$$
Where $\tau = \frac{3}{\mathcal{G}M_\text{obj}v}\left(\frac{\hbar}{m}\right)^2$

In [ ]:
from galpy.orbit import Orbit
from galpy.util import conversion

ro, vo = 8.0, 220.0
# Parameters
GMs = 10.0**6.0 / conversion.mass_in_msol(vo, ro)
r0 = 0.002
vc = 1.0
m = 1e-99
mhbar = (
    conversion.parse_mass(m, ro=ro, vo=vo)
    / conversion._GHBARINKM3S3KPC2
    * ro**2
    * vo**3
)

tau_pred = 3 / (GMs * vc * (mhbar) ** 2)  # analytical orbital time
t = np.linspace(0.0, 2 * tau_pred, 1001)
r_pred = r0 * np.exp(-t / tau_pred)  # analytical solution

from galpy.potential import LogarithmicHaloPotential, FDMDynamicalFrictionForce

Loghalo = LogarithmicHaloPotential(normalize=1.0)
o = Orbit([r0, 0.0, vc, 0.0, 0.0, 0.0])
fdf = FDMDynamicalFrictionForce(GMs=GMs, dens=Loghalo, m=m)
o.integrate(t, Loghalo + fdf, method="dop853_c")

In [ ]:
# check kr value
print("kr value: ", fdf.krValue(r0, vc))

In [ ]:
plt.figure(figsize=(10, 4))

plt.plot(t * conversion.time_in_Gyr(vo, ro), o.R(t) * ro, label="Fuzzy DF integration")
plt.plot(
    t * conversion.time_in_Gyr(vo, ro),
    r_pred * ro,
    label="Fuzzy DF prediction",
    lw=2,
    ls="--",
)
plt.xlabel("t [Gyr]", fontsize=14, fontweight="bold")
plt.ylabel("R [kpc]", fontsize=14, fontweight="bold")
plt.legend(fontsize=12)
plt.tight_layout(pad=2.5)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.3)
plt.gca().set_facecolor("whitesmoke")
plt.grid(True, which="both", linestyle="--", linewidth=0.7, alpha=0.7)

plt.suptitle("Dynamical Friction in NFW halo", fontsize=16, fontweight="bold")
plt.show()